# Analyzing NYC Taxi Fares with RAPIDS

[RAPIDS](https://rapids.ai/) is a suite of GPU accelerated data science libraries with APIs that should be familiar to users of Pandas, scikit-learn, and Dask.

This notebook builds a simple data pipeline to load the data with cuDF (or Pandas), analyze it with cuML (or scikit-learn), find interesting patterns in the data, and build a simple predictive model on top of it.

In [1]:
import glob
import os
import requests

import cudf
import cuml

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

from tqdm.auto import tqdm

/home/ashwin/miniconda3/envs/rapids-gtc2023/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get the data

The main dataset we'll be using in this notebook comes from the [New York City Taxi and Limousine Commision (TLC)](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The TLC publishes data about taxi rides, including pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. The data is made available as PARQUET files, and is published monthly.

The code below downloads the PARQUET files containing trip data for "Yellow" Taxis from the year 2021. It also downloads a second, smaller dataset: a CSV file containing geographical information that will be useful in later parts of our analysis.

In [2]:
def download(url, fname):
    """
    Download file from `url`, writing the result to `fname`.
    If `fname` already exists, do nothing.
    """
    # this code adapted from the tqdm examples
    # https://github.com/tqdm/tqdm/blob/master/examples/tqdm_requests.py
    if os.path.exists(fname):
        return
    response = requests.get(url, stream=True)
    with tqdm.wrapattr(
        open(fname, "wb"),
        "write",
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
        miniters=1,
        desc=fname,
        total=int(response.headers.get("content-length", 0)),
    ) as fout:
        for chunk in response.iter_content(chunk_size=4096):
            fout.write(chunk)


def download_taxi_data(n):
    """
    Download `n` months of taxi data.
    """
    base = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
    fname = "yellow_tripdata_2021-{i:02d}.parquet"
    url = base + fname
    for i in range(1, n + 1):
        download(url.format(i=i), fname.format(i=i))


def download_taxi_zones():
    download(
        "https://gist.githubusercontent.com/shwina/72d79165ce9605d8f6e3378ae717b16b/raw/84a47bc587c99c6736f38a97f9dcc32ba8f89b05/taxi_zones.csv",
        "taxi_zones.csv"
    )


# adjust n between 1-12 depending on the size of analysis
download_taxi_data(n=6)
download_taxi_zones()

In [3]:
!ls

cufile.log			 yellow_tripdata_2021-02.parquet
NYCTaxi.ipynb			 yellow_tripdata_2021-03.parquet
NYCTaxi-JohnZ_224.ipynb		 yellow_tripdata_2021-04.parquet
NYCTaxi-Tutorial-Blank.ipynb	 yellow_tripdata_2021-05.parquet
taxi_zones.csv			 yellow_tripdata_2021-06.parquet
yellow_tripdata_2021-01.parquet


# Part 1: Loading and Preparing Data

The very first operation performed using cuDF initializes the library, which has some overhead. To ensure that this initialization time is not included when we measure code execution time, we "warm up" cuDF before using it for any real work:

In [4]:
_ = cudf.Series([1])  # warmup

### 1.1 Reading the data

First, we'll read the data into a Pandas dataframe using the `pandas.read_parquet()` function. Then, we'll see how to do the same thing with cuDF.

In [5]:
%%time
# TODO: Use Pandas to read all the parquet files into a Pandas DataFrame
# named `df`. Display the result as well as its type.

df = pd.read_parquet(list(sorted(glob.glob("*.parquet"))))
print(type(df))
display(df)

<class 'pandas.core.frame.DataFrame'>


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.00,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.00,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.00,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.00,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.50,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12179185,2,2021-06-30 23:25:00,2021-06-30 23:39:00,NaN,3.27,NaN,None,162,249,0,14.28,0.0,0.5,2.81,0.0,0.3,20.39,NaN,NaN
12179186,2,2021-06-30 23:36:02,2021-07-01 00:05:56,NaN,9.93,NaN,None,217,239,0,32.03,0.0,0.5,7.69,0.0,0.3,43.02,NaN,NaN
12179187,6,2021-06-30 23:06:09,2021-06-30 23:06:45,NaN,7.31,NaN,None,265,76,0,39.50,0.0,0.5,0.00,0.0,0.3,40.30,NaN,NaN
12179188,2,2021-06-30 23:01:24,2021-06-30 23:10:20,NaN,2.83,NaN,None,143,236,0,13.51,0.0,0.5,3.57,0.0,0.3,20.38,NaN,NaN


CPU times: user 4.74 s, sys: 2 s, total: 6.74 s
Wall time: 845 ms


In [6]:
%%time
# Now do the same using cuDF

gdf = cudf.read_parquet(list(sorted(glob.glob("*.parquet"))))
print(type(gdf))
display(gdf)


<class 'cudf.core.dataframe.DataFrame'>


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.00,3.0,0.5,0.00,0.0,0.3,11.80,2.5,<NA>
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.00,0.5,0.5,0.00,0.0,0.3,4.30,0.0,<NA>
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.00,0.5,0.5,8.65,0.0,0.3,51.95,0.0,<NA>
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.00,0.5,0.5,6.05,0.0,0.3,36.35,0.0,<NA>
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.50,0.5,0.5,4.06,0.0,0.3,24.36,2.5,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12179185,2,2021-06-30 23:25:00,2021-06-30 23:39:00,<NA>,3.27,<NA>,<NA>,162,249,0,14.28,0.0,0.5,2.81,0.0,0.3,20.39,<NA>,<NA>
12179186,2,2021-06-30 23:36:02,2021-07-01 00:05:56,<NA>,9.93,<NA>,<NA>,217,239,0,32.03,0.0,0.5,7.69,0.0,0.3,43.02,<NA>,<NA>
12179187,6,2021-06-30 23:06:09,2021-06-30 23:06:45,<NA>,7.31,<NA>,<NA>,265,76,0,39.50,0.0,0.5,0.00,0.0,0.3,40.30,<NA>,<NA>
12179188,2,2021-06-30 23:01:24,2021-06-30 23:10:20,<NA>,2.83,<NA>,<NA>,143,236,0,13.51,0.0,0.5,3.57,0.0,0.3,20.38,<NA>,<NA>


CPU times: user 488 ms, sys: 116 ms, total: 604 ms
Wall time: 602 ms


A few things to note:

- cuDF offers a Pandas-like API. It doesn't require you to learn a new library to take advantage of the GPU.
- cuDF has a `cudf.DataFrame` type that is analogous to `pd.DataFrame`. The primary difference between the two is that `cudf.DataFrame` lives on the GPU and any operations on it utilize the GPU rather than the CPU (and are hence much faster)

### 1.2 Data Cleanup

As usual, the data needs to be massaged a bit before we can start adding features that are useful to an ML model.

1. We'll work with just a subset of columns
2. We'll remove any extraneous spaces from column names and change them to all lowercase
3. We'll cast columns to the appropriate data types
3. For simplicity, we'll replace missing values ("nulls") with a sentinel value -1

The helper function `clean_columns` below does all of the above:

In [7]:
def clean_columns(df, columns_to_keep, column_renames):
    """
    Perform column cleanup on the input DataFrame `df`.
    Drop any columns not present in `columns_to_keep`.
    Then, rename columns according to the mapping `column_renames`.
    Finally, cast any numeric columns from 64-bit to 32-bit data types,
    while filling any nulls that may be present with the sentinel
    value -1.
    """
    # rename columns
    colname_cleanup = {col: col.strip().lower() for col in df.columns}
    df = df.rename(columns=colname_cleanup)
    df = df.rename(column_renames, axis=1)

    # Simplify the payment_type column
    df["is_credit_card"] = df["payment_type"] == 1

    # Drop unwanted columns, and cast data down from
    # 64-bit type to 32-bit type when possible
    for col in df.columns:
        if col not in columns_to_keep:
            print(f"Dropping ({col})")
            df = df.drop(columns=col)
            continue

        # cast int64->int32, float64->float32
        # and fill nulls with -1
        dtype = df[col].dtype
        if dtype.kind in {"i", "f"}:
            if dtype.itemsize == 8:
                df[col] = df[col].astype(dtype.kind + str(dtype.itemsize))
            df[col] = df[col].fillna(-1)

    return df

columns_to_keep = {
    "pickup_datetime",
    "dropoff_datetime",
    "passenger_count",
    "pickup_longitude",
    "pickup_latitude",
    "rate_code",
    "fare_amount",
    "pickup_location",
    "dropoff_location",
    "is_credit_card",
    "airport_fee",
}

column_renames = {
    "tpep_pickup_datetime": "pickup_datetime",
    "tpep_dropoff_datetime": "dropoff_datetime",
    "ratecodeid": "rate_code",
    "pulocationid": "pickup_location",
    "dolocationid": "dropoff_location",
}

In [8]:
%%time

df = clean_columns(df, columns_to_keep, column_renames)
display(df.head())

Dropping (vendorid)
Dropping (trip_distance)
Dropping (store_and_fwd_flag)
Dropping (payment_type)
Dropping (extra)
Dropping (mta_tax)
Dropping (tip_amount)
Dropping (tolls_amount)
Dropping (improvement_surcharge)
Dropping (total_amount)
Dropping (congestion_surcharge)


,pickup_datetime,dropoff_datetime,passenger_count,rate_code,pickup_location,dropoff_location,fare_amount,airport_fee,is_credit_card
0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,1.0,142,43,8.0,-1.0,False
1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,1.0,238,151,3.0,-1.0,False
2,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,1.0,132,165,42.0,-1.0,True
3,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,1.0,138,132,29.0,-1.0,True
4,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,1.0,68,33,16.5,-1.0,True


CPU times: user 4.93 s, sys: 2.79 s, total: 7.72 s
Wall time: 7.62 s


In [9]:
%%time
# TODO: do the same cleanup on `gdf` as we did with `df`

gdf = clean_columns(gdf, columns_to_keep, column_renames)
display(gdf.head())

Dropping (vendorid)
Dropping (trip_distance)
Dropping (store_and_fwd_flag)
Dropping (payment_type)
Dropping (extra)
Dropping (mta_tax)
Dropping (tip_amount)
Dropping (tolls_amount)
Dropping (improvement_surcharge)
Dropping (total_amount)
Dropping (congestion_surcharge)


,pickup_datetime,dropoff_datetime,passenger_count,rate_code,pickup_location,dropoff_location,fare_amount,airport_fee,is_credit_card
0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,1.0,142,43,8.0,-1.0,False
1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,1.0,238,151,3.0,-1.0,False
2,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,1.0,132,165,42.0,-1.0,True
3,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,1.0,138,132,29.0,-1.0,True
4,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,1.0,68,33,16.5,-1.0,True


CPU times: user 88.4 ms, sys: 129 ms, total: 218 ms
Wall time: 287 ms


### 1.3 Filter the data

We'll begin this section by visualizing some of this data using the [Seaborn](https://seaborn.pydata.org/) library. Seaborn does not currently support cuDF DataFrames as input, so we need to use the `.to_pandas()` method to convert to a Pandas DataFrame before passing it to Seaborn.

For a more general guide on how to use RAPIDS with most of the popular visualization libraries, see the viz gallery in our docs: https://docs.rapids.ai/visualization

In [ ]:
sns.boxplot(x="passenger_count", y="fare_amount", data=gdf.to_pandas())

The dataset clearly has some significant outliers. Let's filter the data to throw out some of the outliers and also to remove missing values:

In [ ]:
query_frags = [
    "fare_amount > 0",
    "fare_amount < 500",
    "passenger_count > 0",
    "passenger_count < 6",
]
query = " and ".join(query_frags)
display(query)

In [ ]:
# TODO: apply the query above to the DataFrame `gdf`
# remember to reset the index afterwards.
# (any time you filter the dataframe, it's a good idea to reset its index)

gdf = gdf.query(query)
gdf = gdf.reset_index(drop=True)
# inspect the results of cleaning
display(gdf.head())

In [ ]:
sns.boxplot(x="passenger_count", y="fare_amount", data=gdf.to_pandas())

Now that we've gotten rid of the more obvious outliers, we can continue with the rest of the analysis.

## Part 2: Feature engineering and user-defined functions

We're going to add a few more features to our dataset:

1. Our dataset has the "location IDs" to represent the pickup and dropoff locations. We would like the actual coordinates (latitude and longitude) of the pickup and dropoff locations. Of course, that information will come from another dataset (`taxi_zones.csv`). We'll see how to use cuDF to combine the two datasets.

2. Next, we'll compute the trip distance from the pickup and dropoff coordinates using the [Haversine Distance Formula](https://en.wikipedia.org/wiki/Haversine_formula).

3. Finally, we'll extract additional useful variables from the `pickup_datetime` field using cuDF's datetime functionality.

### 2.1 Combining datasets

The data in `taxi_zones.csv` contains the coordinates for each location. The `'x'` and `'y'` columns refer to the longitute and latitude respectively of each location:

In [ ]:
zones = cudf.read_csv("taxi_zones.csv")
zones

There's something fishy about this dataset though: the `LocationID` column contains non-unique values, meaning that the same location ID maps to more than one zone:

In [ ]:
zones["LocationID"].value_counts()

In [ ]:
zones[(zones["LocationID"] == 103) | (zones["LocationID"] == 56)]

For this demonstration, we'll just drop those location IDs:

In [ ]:
dup_rows = zones[(zones["LocationID"] == 103) | (zones["LocationID"] == 56)]
zones = zones.drop(dup_rows.index, axis=0)
zones = zones.reset_index(drop=True)
zones = zones.set_index("LocationID")
display(zones.head())

One more problem is that not all the pickup and dropoff locations in `gdf` can be found in `zones`:

In [ ]:
gdf["pickup_location"].isin(zones.index).value_counts()

In [ ]:
# TODO: drop any rows in `gdf` where the `pickup_location` or `dropoff_location`
# cannot be found in `zones`. Remember to reset the index of `gdf` afterward.

gdf = gdf[gdf["pickup_location"].isin(zones.index)]
gdf = gdf[gdf["dropoff_location"].isin(zones.index)]
gdf = gdf.reset_index(drop=True)
display(gdf)

In [ ]:
gdf["pickup_location"].isin(zones.index).value_counts()

---

Now we have the coordinate data in `zones` for every pickup and dropoff location in our main dataset `gdf`. Let's use a merge ("join") to combine the two datasets:

In [ ]:
%%time
# TODO: merge `gdf` with `zones`, using the `pickup_location` column
# and the index of `zones` respectively as the merge keys
# the result should contain two columns 'x' and `y`, representing
# the coordinates (longitude and lattitude respectively) of each
# pickup location
gdf.merge(zones, left_on="pickup_location", right_index=True, how="left")

In [ ]:
df = gdf.to_pandas()
zones_cpu = zones.to_pandas()

In [ ]:
%%time
df.merge(zones_cpu, left_on="pickup_location", right_index=True, how="left")

A couple of things to note:

1. The merge operation on the GPU is _fast_ (about 20x faster than CPU).

2. Unlike Pandas, `merge()` in cuDF returns rows in a non-deterministic order. In particular, the original ordering of the join keys is not preserved. We can recover the original ordering by sorting the resulting DataFrame by its index.

Below, we define a helper function that adds both pickup and dropoff coordinates `gdf`, as well as the borough name corresponding to the pickup and dropoff locations (a total of 6 new columns):

In [ ]:
def add_pickup_and_dropoff_info(df, zones):
    pickup_info = (
        df[["pickup_location"]]
        .merge(
            zones[["x", "y", "borough"]],
            left_on="pickup_location",
            right_index=True,
            how="left",
        )
        .sort_index()
    )
    dropoff_info = (
        df[["dropoff_location"]]
        .merge(
            zones[["x", "y", "borough"]],
            left_on="dropoff_location",
            right_index=True,
            how="left",
        )
        .sort_index()
    )
    df["pickup_latitude"] = pickup_info["y"]
    df["pickup_longitude"] = pickup_info["x"]
    df["pickup_borough"] = pickup_info["borough"]
    df["dropoff_latitude"] = dropoff_info["y"]
    df["dropoff_longitude"] = dropoff_info["x"]
    df["dropoff_borough"] = dropoff_info["borough"]
    return df

In [ ]:
gdf = add_pickup_and_dropoff_info(gdf, zones)
display(gdf)

In [ ]:
df = gdf.to_pandas()

---

### 2.2 Computing the trip distance using a user-defined function: `apply()`

The Haversine Distance formula gives the distance between two points on a sphere:

![Haversine distance](https://upload.wikimedia.org/wikipedia/commons/c/cb/Illustration_of_great-circle_distance.svg)

We'd like to use the haversine distance between the pickup and dropoff coordinates as a measure of trip distance. Of course, this is a (bad) approximation, as most taxi trips are not a straight line. But it is enough to yield some useful insights later, and helps illustrate how to use a user-defined function with cuDF.

**Note**: The original dataset does contain the actual trip distance; we filtered it out early in this notebook.

---

First, let's define a function that accepts a single record (row) from our dataframe, and computes the Haversine distance for that trip.

In [ ]:
from math import asin, cos, pi, sin, sqrt


def haversine_distance(row):
    x_1, y_1, x_2, y_2 = (
        row["pickup_latitude"],
        row["pickup_longitude"],
        row["dropoff_latitude"],
        row["dropoff_longitude"],
    )
    x_1 = pi / 180 * x_1
    y_1 = pi / 180 * y_1
    x_2 = pi / 180 * x_2
    y_2 = pi / 180 * y_2

    dlon = y_2 - y_1
    dlat = x_2 - x_1
    a = sin(dlat / 2) ** 2 + cos(x_1) * cos(x_2) * sin(dlon / 2) ** 2

    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers

    return c * r

In [ ]:
%%time
# TODO: Apply the Haversine distance function to each row
# of the Pandas DataFrame `df`, saving the result to a new
# column "h_distance". Hint: use the `apply()` function
# with `axis=1`.
#
# This can take a while to run (a couple of minutes)
df["h_distance"] = df.apply(haversine_distance, axis=1)
display(df.head())

In [ ]:
%%time
# need to remove stselect_dtypesg columns before using `apply()`
# in cuDF
gdf_numeric = gdf.select_dtypes(exclude="object")
gdf["h_distance"] = gdf_numeric.apply(haversine_distance, axis=1)
gdf.head()

The call to `apply()` is much faster on the GPU.

In fact, much of the time spent in the call to `apply()` is in JIT compilation. If you run the UDF again, we'll see that it takes a very small fraction of time compared to the first run.

---

For more about user-defined functions in cuDF, features and limitations, see the [cuDF documentation](https://docs.rapids.ai/api/cudf/stable/user_guide/guide-to-udfs.html).

For advanced GPU-accelerated spatial calculations, check out [cuSpatial](https://medium.com/rapids-ai/releasing-cuspatial-to-accelerate-geospatial-and-spatiotemporal-processing-b686d8b32a9).


### 2.3 Extracting datetime features

Finally, we'll use the datetime functionality provided by cuDF to extract some features from the `pickup_datetime` column:

In [ ]:
def add_datetime_features(df):
    df["hour"] = df["pickup_datetime"].dt.hour
    df["year"] = df["pickup_datetime"].dt.year
    df["month"] = df["pickup_datetime"].dt.month
    df["day"] = df["pickup_datetime"].dt.day

    df["day_of_week"] = df["pickup_datetime"].dt.dayofweek

    df = df.drop(columns=["pickup_datetime", "dropoff_datetime"])

    df["is_weekend"] = (df["day_of_week"] >= 5).astype(np.int32)
    return df

In [ ]:
gdf = add_datetime_features(gdf)

In [ ]:
display(gdf.tail())

In [ ]:
df = gdf.to_pandas()

# Section 3: Exploratory Analysis and Machine Learning

Let's say we're studying consumer behavior on behalf of the taxi commission. Are there a couple of clear "types" of rides that come up again and again? What are some of the key patterns we see in the data? Can we reliably predict some elements of user behavior?

Let's just do this on GPU, because it's faster and the code only differs by using `cuml.` instead of `sklearn.`

In [ ]:
%matplotlib inline

### 3.1 Unsupervised learning - UMAP

We'll use the UMAP algorithm to explore the data in an unsupervised manner. UMAP takes high-dimensional data, like our taxi dataset with 20 features, and maps it down to a lower dimensional space so we can view all of the points together in a 2d plot while preserving much of the structure of the dataset. This means that similar points will be clearly grouped together in the plot and we can see how patterns change across 

In [ ]:
%%time

# cuML can handle the full dataset nicely, but it's hard to visualize a million points sometimes
# so let's start with a subset. Note that doing UMAP over 100k points on CPU can easily take minutes
gdf_sample = gdf.sample(100_000).reset_index()

# Fit
umap = cuml.manifold.UMAP()
umap_out = umap.fit_transform(
    cuml.preprocessing.RobustScaler().fit_transform(
        gdf_sample.drop(["pickup_borough", "dropoff_borough"], axis=1)
    )
)

gdf_sample["umap_x"] = umap_out.iloc[:, 0]
gdf_sample["umap_y"] = umap_out.iloc[:, 1]

In [ ]:
import holoviews as hv
import hvplot.cudf

# Let's use hvplot to plot our UMAP results along with some useful ancillary data
# Because hvplot has native support for cudf (via hvplot.cudf), we don't have to do any conversion
gdf_sample.hvplot.scatter(
    x="umap_x",
    y="umap_y",
    color=["dropoff_borough"],
    groupby=["day_of_week"],
    hover_cols=[
        "pickup_latitude",
        "pickup_longitude",
        "pickup_borough",
        "airport_fee",
        "h_distance",
        "fare_amount",
        "pickup_location",
        "is_credit_card",
        "dropoff_borough",
    ],
    s=0.5,
).opts(title="Taxi Data", width=700, height=500, cmap="Category10")

### 3.2 Build a supervised model to predict payment form

Let's imagine you're a taxi operator trying to understand consumer payment behavior. What types of rides will be paid by credit card and what types via cash? Let's build a predictive model to help us understand.

#### Splitting train and test data

We'll want to know how accurate our model is, so let's start by splitting the dataset into "train" and "test" subsets randomly.

In [ ]:
%time

# First let's split using Pandas and scikit-learn

y_df = df["is_credit_card"]
X_df = df.drop(columns=["is_credit_card", "pickup_borough", "dropoff_borough"])

# Split our dataframes
X_train_df, X_test_df, y_train_df, y_test_df = sklearn.model_selection.train_test_split(
    X_df, y_df
)

# Create array versions of these dataframes
X_train_np, X_test_np = (
    X_train_df.to_numpy(np.float32),
    X_test_df.to_numpy(np.float32),
)
y_train_np, y_test_np = (
    y_train_df.to_numpy(np.float32),
    y_test_df.to_numpy(np.float32),
)

len(X_train_df)

When we think of ML, we often think of running large predictive models. But scikit-learn also provides a huge toolbox of utilities like preprocessing, data splitting, and more that are essential for ML practitioners. cuML contains GPU-based analogues of these utilities so that your data does not need to take a round trip back to CPU just to partition it.

In [ ]:
%%time

# Now do the same on GPU

y_gdf = gdf["is_credit_card"]
X_gdf = gdf.drop(columns=["is_credit_card", "pickup_borough", "dropoff_borough"])

# Split our dataframes
(
    X_train_gdf,
    X_test_gdf,
    y_train_gdf,
    y_test_gdf,
) = cuml.model_selection.train_test_split(X_gdf, y_gdf)

# Create array versions of these dataframes
X_train_gpu, X_test_gpu = (
    X_train_gdf.to_cupy(np.float32),
    X_test_gdf.to_cupy(np.float32),
)
y_train_gpu, y_test_gpu = (
    y_train_gdf.to_cupy(np.float32),
    y_test_gdf.to_cupy(np.float32),
)

len(X_train_gdf)

#### Fit a simple supervised model with cuML

cuML supports a large range of supervised models, all emulating the scikit-learn interfaces. See the README (https://github.com/rapidsai/cuml) for a recent list. Here, we'll try a very common but powerful model - a random forest ensemble for classification. Specifically, we want to classify whether the trip was paid for by credit card (the positive class) or not.

In [ ]:
from cuml.ensemble import RandomForestClassifier as cuRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier as skRandomForestClassifier

In [ ]:
%%time

# Scikit-learn will parallelize over all CPU cores with n_jobs=-1

sk_model = skRandomForestClassifier(n_estimators=250, n_jobs=-1)
sk_model.fit(X_train_np, y_train_np)

In [ ]:
%%time

# TODO: Build a similar model on GPU with cuML
# This is RandomForest estimator builds 250 separate trees, each of which tries to predict whether or
# not a transaction uses a credit card, then averages the results together. Note again that its' the same
# API as sklearn but significantly faster. These speedups typically get greater as the dataset grows.

cuml_model = cuRandomForestClassifier(n_estimators=250)
cuml_model.fit(X_train_gpu, y_train_gpu)

In [ ]:
# Let's use the model to predict from the test set and evaluate the predictions' accuracy

rf_predictions = cuml_model.predict(X_test_gpu)

# Compute the probability that the transaction used a credit card, according to our model
rf_probabilities = cuml_model.predict_proba(X_test_gpu)[:, 1]

## Just as cuML provides utilities for data preprocessing and splitting, it also provides a
## wide set of GPU-accelerated metrics, like accuracy and AUC score. These are fast and
## ensure you don't need to fall back to CPU for any of your pipeline.
print("Accuracy: ", cuml.metrics.accuracy.accuracy_score(y_test_gpu, rf_predictions))
print("AUC: ", cuml.metrics.roc_auc_score(y_test_gpu, rf_probabilities))

In [ ]:
rf_probabilities

In [ ]:
# Ok, so we're right abuot 75% of the time about whether the transaction will use a credit card
# The AUC (area under the curve) statistic shows us in a bit more detail that our model has value
# but isn't particularly amazing.
#
# Let's actually plot the distributions of our output probabilities to get a feel whether
# we're really outputting high probabilities for transactions that used credit cards

sns.kdeplot(x=rf_probabilities.get())  # hue=y_test_gpu.get())